### 그레이디언트 부스팅
- 트리 앙상블 알고리즘 중 성능이 좋아서 구현한 패키지들이 많다
  LinghtGBM, CatBoost, XGBoost
- 테이블 형식 데이터에 대한 예측에서 엄청난 성능을 보임
- 경사 하강법(GD-Gradient Descent)에서의 경사와 유사(손실(loss)함수로 최저점 찾기)
- 손실 함수- 회귀(MSE), 분류(로지스틱손실)
- Boosting이란 약한 분류기를 결합하여 강항 분류기를 반드는 과정 - 트리를 순차적으로 만들어간다 -> 속도가 늦을수도 있음

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, random_state=42, test_size=0.2)

In [2]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
np.mean(scores['train_score']), np.mean(scores['test_score'])

(0.8881086892152563, 0.8720430147331015)

In [6]:
# 트리 개수 확대  - 학습률 설정
gb = GradientBoostingClassifier(n_estimators =500, learning_rate =0.2, random_state =42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

#훈련 후 특징 중요도 출력
gb.fit(train_input, train_target)
gb.feature_importances_

0.9464595437171814 0.8780082549788999


array([0.15872278, 0.68011572, 0.16116151])

In [8]:
# 히스토그램 기반 그레이디언트 부스팅
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)
np.mean(scores['train_score']), np.mean(scores['test_score'])

(0.9321723946453317, 0.8801241948619236)

In [13]:
# 히스토그램 권장 방법
# permutation(순열, 치환)방식으로 특성
from sklearn.inspection import permutation_importance
hgb.fit(train_input, train_target)
res = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)
# 특성을 하나씩 랜덤하게 섞어 모델의 성능 뱐화 관찰 -> 결과값의 loss가 커질 경우 변수는 중요한 변수라는 것을 의미
# n_repeats매개변수는 랜덤하게 섞을 횟수를 지정(기본은 5)

res.importances_mean

res = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)
res.importances_mean

hgb.score(test_input, test_target)

0.8723076923076923

### XGBoost vs LightGBM
- 그레디언트부스팅 전용 오픈소스 라이브러리
- XGBoost는 캐글에서 사용되면서 인기가 높아짐
- LightGBM은 MS사에서 개발, 히스토그램기반부스팅 지원